In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve


In [63]:
# Fixing the directory path 
import os
os.chdir("C:/Users/RAZER BLADE/Downloads/AI Path/Multi-Class-Classification")

In [64]:
data = pd.read_csv("Datasets/train.csv")

# Basic Preprocessing

In [46]:
df = data.copy()

# Drop the age column 
df = df.drop(['Age'], axis=1)

# Replace the numerical value of Hepatomegaly 
df.loc[df['Hepatomegaly'] == '119.35', 'Hepatomegaly'] = df['Hepatomegaly'].mode()[0]

In [65]:
Y = df['Status']
X = df.drop(['Status', 'id'], axis=1)
print(X.shape, Y.shape)

(15000, 17) (15000,)


In [123]:
df['Stage'] = df['Stage'].astype('int')
df['Stage']

0        3
1        4
2        2
3        3
4        3
        ..
14995    3
14996    4
14997    4
14998    4
14999    4
Name: Stage, Length: 15000, dtype: int32

In [125]:
from sklearn.model_selection import train_test_split

X_train,  X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=8)
print('Trainning Shape', X_train.shape, Y_train.shape)
print('Trainning Shape', X_test.shape, Y_test.shape)

Trainning Shape (12000, 17) (12000,)
Trainning Shape (3000, 17) (3000,)


In [126]:
Y_train.value_counts()

Status
C     8053
D     3634
CL     313
Name: count, dtype: int64

# Splitting Variables into Numerical and categorical 

In [127]:
numerical_cols = X_train.select_dtypes(include=['float', 'int']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

X_train_num = X_train[numerical_cols]
X_train_cat = X_train[categorical_cols]

X_test_num = X_test[numerical_cols]
X_test_cat = X_test[categorical_cols]

In [152]:
categorical_cols

Index(['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema'], dtype='object')

# Nan Values

In [128]:
# replacing the median and mode 
def imputation_statistics(df_fill):
    """ impute null values with mode for categorical variables and median for numerical ones
    """
    for col in df_fill.columns:
        if df_fill[col].isna().any():
            df_fill[col] = df_fill[col].fillna(df_fill[col].mode()[0])
    return df_fill
#X_train_m = imputation_statistics(X_train)
#X_test_m = imputation_statistics(X_train)

In [129]:
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.linear_model import LinearRegression


# Numerical Variables 
Imp = IterativeImputer(estimator=LinearRegression(), max_iter=10, random_state = 0)
Imp.fit(X_train_num)

X_train_num_imputed = Imp.transform(X_train_num)
X_test_num_imputed = Imp.transform(X_test_num)

X_train_num_imputed = pd.DataFrame(X_train_num_imputed)
X_train_num_imputed.columns = X_train_num.columns
X_test_num_imputed = pd.DataFrame(X_test_num_imputed)
X_test_num_imputed.columns = X_test_num.columns

# Categorical variables
X_train_cat_imputed = imputation_statistics(X_train_cat)
X_test_cat_imputed = imputation_statistics(X_test_cat)


In [ ]:
import sys
from dataclasses import dataclass

import numpy as np 
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.linear_model import LinearRegression

from src.exception import CustomException
from src.logger import logging
import os

from src.utils import save_object

# 1. Create a data transfo config to configure path 
@dataclass
class DataTransformationConfig:
    preprocessor_obj_file_path = os.path.join('artifact', "preprocessor.pkl")
    
# Creating Data transfo class 
class DataTransformation:
# Initialise the path config within the init
    def __init__(self):
        self.data_transformation_config = DataTransformationConfig()
    
    def imputing():
        imputer = IterativeImputer(estimator=LinearRegression(), max_iter=10, random_state = 0)
        Imp.fit(X_train_num)
        X_train_num_imputed = Imp.transform(X_train_num)
        X_test_num_imputed = Imp.transform(X_test_num)

        X_train_num_imputed = pd.DataFrame(X_train_num_imputed)
        X_train_num_imputed.columns = X_train_num.columns
        X_test_num_imputed = pd.DataFrame(X_test_num_imputed)
        X_test_num_imputed.columns = X_test_num.columns

        def imputation_statistics(df_fill):
            for col in df_fill.columns:
                if df_fill[col].isna().any():
                    df_fill[col] = df_fill[col].fillna(df_fill[col].mode()[0])
        return df_fill

        X_train_cat_imputed = imputation_statistics(X_train_cat)
        X_test_cat_imputed = imputation_statistics(X_test_cat)




    def get_data_transformer_object(self):
        try:
            numerical_columns = ['N_Days', 'Bilirubin', 'Cholesterol', 
                                 'Albumin', 'Copper', 'Alk_Phos',
                                'SGOT', 'Tryglicerides', 'Platelets', 
                                'Prothrombin', 'Stage']
            categorical_columns = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
           

            num_pipeline = Pipeline(
                steps=[
                ("imputer", IterativeImputer(estimator=LinearRegression(), max_iter=10, random_state = 0)),
                ("scaler", RobustScaler())

                ]
            )

            cat_pipeline = Pipeline(

                steps=[
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("one_hot_encoder", OneHotEncoder())
                
                ]

            )

            logging.info(f"Categorical columns: {categorical_columns}")
            logging.info(f"Numerical columns: {numerical_columns}")

            preprocessor = ColumnTransformer(
                [
                ("num_pipeline",    num_pipeline,   numerical_columns),
                ("cat_pipelines",   cat_pipeline,  categorical_columns)

                ]


            )

            return preprocessor
        
        except Exception as e:
            raise CustomException(e,sys)
        
    def initiate_data_transformation(self, train_path, test_path):

        try:
            # Read the train. test that will come from data_ingestion
            train_df = pd.read_csv(train_path)
            test_df = pd.read_csv(test_path)
            logging.info("Read train and test data completed")

            # Creating the preprocessor
            preprocessing_obj = self.get_data_transformer_object()
            logging.info("Obtaining preprocessing object")

            # Preparing the X_train Y_train X_test Y_test
            target_column_name = "Status"
            numerical_columns = ['N_Days', 'Bilirubin', 'Cholesterol', 
                                 'Albumin', 'Copper', 'Alk_Phos',
                                'SGOT', 'Tryglicerides', 'Platelets', 
                                'Prothrombin', 'Stage']
            categorical_columns = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
            columns_delete = ['id', 'Age']

            # Train    
            X_train = train_df.drop(columns=columns_delete + [target_column_name], axis=1)
            Y_train = train_df[target_column_name]

            # Test 
            X_test = test_df.drop(columns=columns_delete + [target_column_name], axis=1)
            Y_test = test_df[target_column_name]

            # SPLITTING Categorical and numerical 
            numerical_cols = X_train.select_dtypes(include=['float', 'int']).columns
            categorical_cols = X_train.select_dtypes(include=['object']).columns
            # Train 
            X_train_num = X_train[numerical_cols]
            X_train_cat = X_train[categorical_cols]
            # Test 
            X_test_num = X_test[numerical_cols]
            X_test_cat = X_test[categorical_cols]

            logging.info(
                f"Applying preprocessing object on training dataframe and testing dataframe."
            )

            # fit the preprocessor and transform train and test data 
            X_train_ = preprocessing_obj.fit_transform(input_feature_train_df)
            X_test_ = preprocessing_obj.transform(input_feature_test_df)

            #  concatenate arrays features and target along the second axis (i.e., columns)
            train_arr = np.c_[input_feature_train_arr, np.array(target_feature_train_df)]
            test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]

            
            logging.info(f"Saving the preprocessing object.")
            save_object(

                file_path = self.data_transformation_config.preprocessor_obj_file_path,
                obj = preprocessing_obj

            )
            logging.info(f"Saved ! preprocessing object.")

            return (
                train_arr,
                test_arr,
                self.data_transformation_config.preprocessor_obj_file_path,
            )
        except Exception as e:
            raise CustomException(e,sys)





# Feature Engineering

### Encoding 

In [130]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import pandas as pd

def encoding_mixed(X_train_cat_imputed, X_test_cat_imputed, onehot_cols, ordinal_cols):
    """
    Encodes categorical data using one-hot encoding for some variables and ordinal encoding for others.

    Args:
        X_train_cat_imputed: DataFrame containing imputed categorical features of the training set.
        X_test_cat_imputed: DataFrame containing imputed categorical features of the test set.
        onehot_cols: List of column names to apply one-hot encoding.
        ordinal_cols: List of column names to apply ordinal encoding.

    Returns:
        X_train_encoded: Encoded training set as a DataFrame.
        X_test_encoded: Encoded test set as a DataFrame.
    """
    # Define the transformers for one-hot and ordinal encoding
    transformers = [
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), onehot_cols),
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ordinal_cols)
    ]

    # Create the ColumnTransformer
    column_transformer = ColumnTransformer(transformers, remainder='passthrough')

    # Fit the transformer on the training set and transform both datasets
    X_train_encoded = column_transformer.fit_transform(X_train_cat_imputed)
    X_test_encoded = column_transformer.transform(X_test_cat_imputed)

    # Get feature names for one-hot encoded columns
    onehot_feature_names = column_transformer.named_transformers_['onehot'].get_feature_names_out(onehot_cols)

    # Combine the one-hot and ordinal column names
    all_columns = list(onehot_feature_names) + ordinal_cols + [
        col for col in X_train_cat_imputed.columns if col not in onehot_cols + ordinal_cols
    ]

    # Convert to DataFrame with proper column names
    X_train_encoded = pd.DataFrame(X_train_encoded, columns=all_columns, index=X_train_cat_imputed.index)
    X_test_encoded = pd.DataFrame(X_test_encoded, columns=all_columns, index=X_test_cat_imputed.index)

    return X_train_encoded, X_test_encoded

# Usage
onehot_cols = ['Ascites', 'Edema']  # Replace with your one-hot encoded column names
ordinal_cols = ['Drug', 'Sex', 'Hepatomegaly','Spiders' ]  # Replace with your ordinal encoded column names

X_train_cat_encoded, X_test_cat_encoded = encoding_mixed(X_train_cat_imputed, X_test_cat_imputed, onehot_cols, ordinal_cols)


In [131]:
def target_encoding(Y):
    code = {'C':1, 'D':0, 'CL':2}
    Y = Y.map(code)
    return Y

In [132]:
Y_train = target_encoding(Y_train)
Y_test = target_encoding(Y_test)

In [134]:
Y_train.value_counts()

Status
1    8053
0    3634
2     313
Name: count, dtype: int64

# Scaling 

In [135]:
from sklearn.preprocessing import RobustScaler
def scaling(X_train, X_test):
    X_train_tf = X_train.copy()
    X_test_tf = X_test.copy()
    numerical_features = [col for col in X_train.select_dtypes('float').columns if col != 'Stage']
    #only scale numeric varaibles in this case rather than the dummy variables for categories 
    rob = RobustScaler()
    X_train_tf.loc[:, numerical_features] = rob.fit_transform(X_train_tf.loc[:, numerical_features])
    X_test_tf.loc[:, numerical_features] = rob.transform(X_test_tf.loc[:, numerical_features])
    return X_train_tf, X_test_tf

X_train_tf, X_test_tf = scaling(X_train_num_imputed, X_test_num_imputed) 

# Final merge of preprocessed categorical and numerical features

In [136]:
# concat 
X_train_final = pd.concat([X_train_tf.reset_index().drop('index', axis=1), X_train_cat_encoded.reset_index().drop('index', axis=1)], axis=1)
X_test_final = pd.concat([X_test_tf.reset_index().drop('index', axis=1), X_test_cat_encoded.reset_index().drop('index', axis=1)], axis=1)

In [137]:
X_train_final['Stage'] = X_train_final['Stage'].astype('int')

In [138]:
X_train_final

,N_Days,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,...,Ascites_D-penicillamine,Ascites_N,Ascites_Y,Edema_N,Edema_S,Edema_Y,Drug,Sex,Hepatomegaly,Spiders
0,0.125644,-0.166667,0.190400,-0.604167,-0.086568,0.086999,-0.059866,0.243489,0.231405,-0.777778,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.235825,0.083333,-0.179641,-1.187500,0.179954,-0.066618,0.245873,0.236141,-0.933884,-0.777778,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.610825,2.166667,-0.662348,-0.791667,8.901006,7.749653,0.812009,-1.765576,-0.371901,0.555556,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,0.567655,-0.083333,-0.781231,0.062500,-0.164399,-0.301297,2.115715,-2.703076,0.735537,0.000000,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
4,-0.704897,0.750000,-0.037993,0.041667,0.178978,-0.197752,0.196841,0.030341,-0.909091,-0.222222,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,1.313144,-0.166667,-1.375645,0.145833,-0.704565,0.431819,0.113595,2.484424,0.429752,-1.111111,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11996,1.452320,0.000000,-0.849164,0.937500,-1.056848,-1.094396,-1.394078,-0.703076,-0.198347,1.111111,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11997,-0.269330,-0.083333,-0.361900,-0.375000,-0.045774,-0.138230,0.034627,-0.098578,-0.652893,-0.222222,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
11998,0.329897,-0.166667,-0.271732,-0.791667,-0.305312,10.135347,-0.929069,0.296924,2.570248,0.111111,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Evaluation metrics 

In [139]:
# log loss function 
def log_loss(Y_test, y_pred_proba):
    """
    Calcule l'opposé de la log-vraisemblance pour un problème de classification multiclasse.
    
    :param Y_test: ndarray de shape (N, M), les vérités terrain encodées en one-hot (1 pour la classe vraie, 0 sinon).
    :param y_pred: ndarray de shape (N, M), les probabilités prédites pour chaque classe.
                   Chaque ligne doit contenir les probabilités non normalisées.
    :return: float, la log-vraisemblance négative normalisée.
    """
    # Normaliser les probabilités
    y_pred_proba = y_pred_proba / np.sum(y_pred_proba, axis=1, keepdims=True)
    
    # Éviter les extrêmes du logarithme
    epsilon = 1e-15
    y_pred_proba = np.clip(y_pred_proba, epsilon, 1 - epsilon)

    N = Y_test.shape[0]
    M = y_pred_proba.shape[1]
    y_one_hot = np.zeros((N, M))
    y_one_hot[np.arange(N), Y_test] = 1
    
    # Calcul de la log-vraisemblance
    log_loss = -np.sum(y_one_hot * np.log(y_pred_proba)) / Y_test.shape[0]
    
    return log_loss

In [140]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

def evaluate_model(Y_test, Y_pred, y_pred_proba):

    loglosss = log_loss(Y_test, y_pred_proba)
    accuracy = accuracy_score(Y_test, Y_pred)
    precision = precision_score(Y_test, Y_pred, average='micro')
    recall = recall_score(Y_test, Y_pred, average='micro')
    f1 = f1_score(Y_test, Y_pred, average='micro')
    
    return accuracy, precision, recall, f1, loglosss

Model Trainer

In [141]:
# Initialize the models
from xgboost import XGBClassifier
models = {
    'k-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),  # Default k=5
    'Decision Tree': DecisionTreeClassifier(max_depth=None, random_state=42),
    'Naive Bayes': GaussianNB(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42),
    'xgboost': XGBClassifier( n_estimators=100, max_depth=200)
}

In [142]:
def trainning_models(models, X_train, Y_train, X_test, Y_test):
    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, Y_train) # Train model

        # Make predictions
        Y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)

        # Evaluate Test dataset
        accuracy, precision, recall, f1, logloss = evaluate_model(Y_test, Y_pred, y_pred_proba)
        print(model)
        print("Accuracy: {0}".format(accuracy))
        print("Precision: {0}".format(precision))
        print("Recall: {0}".format(recall))
        print("f1: {0}".format(f1))
        print("logloss: {0}".format(logloss))
        
        # Confusion Matrix
        cm = confusion_matrix(Y_test, Y_pred)
        plt.figure()
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=list(Y_test.unique()), yticklabels=list(Y_test.unique()))
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title('Confusion Matrix')
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.gca().xaxis.set_label_position('top')  # Move x-axis label to the top
        plt.gca().xaxis.tick_top()                # Move x-axis ticks to the top
        plt.show()

        print('='*35)
        print('\n')

In [ ]:
trainning_models(models, X_train_final, Y_train.to_numpy(), X_test_final, Y_test)

Tree Based models are the best since our Dataset has a lot of outliers

# Enhancing model's performance with Data processing techniques 

In [145]:
def model_fit(X_new, Y_train, X_test_final):
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
    model.fit(X_new, Y_train) # Train model

    # Make predictions
    Y_pred = model.predict(X_test_final[X_new.columns])
    y_pred_proba = model.predict_proba(X_test_final[X_new.columns])

    # Evaluate Test dataset
    accuracy, precision, recall, f1, logloss = evaluate_model(Y_test, Y_pred, y_pred_proba)
    print("logloss: {0}".format(logloss))

In [147]:
Y_train.value_counts()

Status
1    8053
0    3634
2     313
Name: count, dtype: int64

In [149]:
from imblearn.over_sampling import SMOTE 
smote = SMOTE(sampling_strategy = {1:8053, 2:1000, 0:3634} , random_state = 42) 
X_smote, y_smote = smote.fit_resample(X_train_final ,Y_train)

model_fit(X_smote, y_smote, X_test_final)

logloss: 0.40878170455654983


In [96]:
from sklearn.feature_selection import (
    SelectKBest, 
    chi2, 
    f_classif, 
    f_regression,
    r_regression,
    mutual_info_classif,
    mutual_info_regression
)

from sklearn.feature_selection import SelectKBest, f_classif

kb = SelectKBest(f_classif, k=15)
X_new = kb.fit_transform(X_train_final, Y_train)
X_new = pd.DataFrame(X_new)
X_new.columns = kb.get_feature_names_out()



Status
1    8053
2    3634
0    3634
Name: count, dtype: int64

Balancing Classes : 